## Prediction of Breast Cancer Severity with Multi-Omics Data

Can the QLattice reveal hidden mechanisms underlying breast cancer severity using multi-omics data (mutations, copy numbes, gene expression and protein levels)? 

![Breast Cancer Paper](https://els-jbs-prod-cdn.jbs.elsevierhealth.com/cms/attachment/de466e25-32e8-48dd-8f69-cee69e28b345/fx1.jpg)

**The Dataset**

* 705 breast tumour samples (611 patients survived, 94 patients died)



Four Data Types (n features):
* Copy Number Variations (860)
* Somatic Mutations (249)
* Gene Expression (604)
* Protein Expression (223)

**Total: 1936 features**

![The Data Types](https://d1g9yur4m4naub.cloudfront.net/image-handler/ts/20210121043242/ri/673/picture/2021/1/shutterstock_695774158.jpg)

The dataset is described in detail in the paper: https://www.cell.com/cell/fulltext/S0092-8674(15)01195-2.

# Python imports
In this notebook we will use only three python modules: the `feyn` module to access the QLattice, and the `pandas` module to access the data, and sklearn to split the data in test/train sets

In [ ]:
#The feyn Python module is not installed on Kaggle by default so we have to pip install it first. 
#__Note__: the pip install will fail unless you enable *Internet* in the *settings* to the right --->
!pip install feyn

In [ ]:
import feyn
import pandas as pd
import sklearn.model_selection

## Load the dataset

In [ ]:
data = '/kaggle/input/brca-multiomics-tcga/data.csv'
df = pd.read_csv(data)
df

# Target balance
Let's have a quick look at the balance of target variable

In [ ]:
df["vital.status"].value_counts()

The target variable is pretty unbalanced. Let's make sure we account for that when we split our data into train and test. We want to make sure to have the same ratio of cases/controls in both.

In [ ]:
train, test = sklearn.model_selection.train_test_split(df,stratify=df["vital.status"], train_size=.66, random_state=42)

# Allocate a QLattice
The actual QLattice is a quantum simulator that runs on Abzu's hardware, but we can allocate one to use for our analysis with a single line of code. Hopefully the following line will get us one.

In [ ]:
ql = feyn.connect_qlattice()

# Search for the best model

We are now ready to instruct the QLattice to search for the best mathematical model to explain the data. Here we use the high-level convenience function that does everything with sensible defaults: https://docs.abzu.ai/docs/guides/essentials/auto_run.html.

For more detailed control, we could use the primitives: https://docs.abzu.ai/docs/guides/primitives/using_primitives.html

### Let's start simple. We constrain the model to have 3 edges (e.g. 2 features and one interaction)
Essentially, we're asking the question "which two features best describe my data and how do they relate to eachother?".

In [ ]:
ql.reset(random_seed=42)
models = ql.auto_run(train, output_name="vital.status", kind="classification", max_complexity=3, n_epochs=25)

In [ ]:
models[0].plot(train, test)

This is already a pretty good model given that a random forest only scores around AUC=0.66 (using hundreds of features and not being interpretable).

### Let's see what's going on in the model - Partial Plots

In [ ]:
# Training Data
models[0].plot_partial2d(train)

In [ ]:
#Test Data
models[0].plot_partial2d(test)

Nice, the model generalises well on unseen data!

### Looking at the ROC curve

In [ ]:
models[0].plot_roc_curve(test)

### Let's look at some other machine-learning methods

In [ ]:
rf = feyn.reference.RandomForestClassifier(train, output_name="vital.status", random_state = 42)
gb = feyn.reference.GradientBoostingClassifier(train, output_name="vital.status", random_state = 42)
lr = feyn.reference.LogisticRegressionClassifier(train, output_name="vital.status", max_iter=10000, random_state = 42)

rf.plot_roc_curve(test, label="Random Forest")
gb.plot_roc_curve(test, label="Gradient Boosting")
lr.plot_roc_curve(test, label="Logistic Regression")

## The Multi-Omics Capabilities of the QLattice

### What happens when I let the QLattice explore for a while?
In this example I'm loading in a model that was trained for 200 iterations (using criterion = "bic" and max_complexity = 7). This took around 30 minutes on my machine (M1 Macbook Air 2021).

In [ ]:
multi_model = feyn.Model.load("/kaggle/input/brca-multiomics-tcga/model_200its_md2_bic_0.model")

In [ ]:
multi_model.plot(train, test)

Nice AUC! Interestingly, this model includes three different data types: Gene expression (rs), copy numbers (cn) and mutations (mu).
We didn't even have to do anything special to the data, the QLattice figured out the correct normalisations itself!

### Looking at people without TNXB mutations
In people **without** TNXB mutations both high APOB and VWDE gene expression are associated with death.

In [ ]:
multi_model.plot_partial2d(df, fixed = {"mu_TNXB" : 0, "cn_ANKRD30B" : -1})

### Looking at TNXB mutation carriers
In people **with** a TNXB mutation only high APOB is required for a case to be fatal.

In [ ]:
multi_model.plot_partial2d(df, fixed = {"mu_TNXB" : 1, "cn_ANKRD30B" : -1})

The QLattice identified a **genetic switch**, i.e., a mutation in a gene (TNXB) that seems to drive cancer severity. In the top figure, we show the decision boundary for non-TNXB mutation carriers: Here, individuals with high APOB and VWDE gene-expression seem to be at risk of dying.
In the bottom figure, we show the predictions for TNXB-mutation carriers. Here, high levels of APOB are predicted to be detrimental, no matter the levels of VWDE.


## Conclusion

The QLattice seamlessly provides insights into complex datasets and outperforms traditional machine-learning methods.